In [1]:
!pip install scrapy

    100% |████████████████████████████████| 256kB 1.9MB/s ta 0:00:01
    100% |████████████████████████████████| 3.0MB 213kB/s eta 0:00:01
    100% |████████████████████████████████| 5.9MB 109kB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 6.6MB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 5.4MB/s eta 0:00:01
    100% |████████████████████████████████| 153kB 4.0MB/s eta 0:00:01
  Running setup.py bdist_wheel for Twisted ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/b3/76/f7/85353c829c0881e74b5366ce0ed59042b098bb4903e2da8828
  Running setup.py bdist_wheel for PyDispatcher ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
  Running setup.py bdist_wheel for zope.interface ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/c6/b2/d2/be6785a207eaa58d76debc10c9d5c66196b40a88abb61d6af7
Successfully built Twisted PyDispatcher zope.interface
You are using pip 

In [2]:
!pip install beautifulsoap4

  Could not find a version that satisfies the requirement beautifulsoap4 (from versions: )
No matching distribution found for beautifulsoap4
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip install bs4

    100% |████████████████████████████████| 92kB 2.4MB/s ta 0:00:01
  Running setup.py bdist_wheel for bs4 ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!pip install lxml

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup
import os

In [2]:
class SpidyQuotesSpider(scrapy.Spider):
    name = 'spidyquotes'
    
    quotes_base_url = 'https://www.exportcenter.ru/events/?AJAX_MODE=1&PAGEN_1=%s&date_period=all&city=&search=&rec_participation=&reg_open=Y&international='
    start_urls = [quotes_base_url % 1]
    download_delay = 1.5

    def parse(self, response):
        data = []
        if (os.path.exists("log.txt")):
            with open("log.txt") as f:
                for line in f:
                    tmp = line
                    data.append(tmp)
        
        html = response.text
        soup = BeautifulSoup(html, 'lxml')
        news = soup.find_all('div', class_='event-block')
        
        for new in news:
            url = 'https://www.exportcenter.ru' + new.find('div', class_='event-item__desc').find('a').get('data-url')
            name = new.find('a', class_ = 'event-item__title js-events-detail-link').getText().replace('  ','').replace('\n','').replace('\r','')
            date = new.find('span', class_ = 'event-item__date_day').getText() + '/' + new.find('span', class_ = 'event-item__date_month').getText()
            
            is_ = False
            for i in range(len(data)):
                if name in data[i]:
                    is_ = True
                    break
            
            if is_ == False: 
                with open('log.txt', 'a') as f:
                    f.write('name: {0}, url: {1}, date: {2}\n'.format(name, url, date))
                yield {
                    'name': name,
                    'url': url,
                    'date': date,
                }
            
        if soup.find('div', class_='event-btn_more js-events-go-next'):
            next_page = int(soup.find('div', class_='event-btn_more js-events-go-next').get('data-next-page'))
            yield scrapy.Request(self.quotes_base_url % next_page)

if __name__ == "__main__":
    process = CrawlerProcess()

    process.crawl(SpidyQuotesSpider)
    process.start() # the script will block here until the crawling is finished
    process.join()

2018-07-02 04:00:47 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: scrapybot)
2018-07-02 04:00:47 [scrapy.utils.log] INFO: Versions: lxml 4.2.3.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.4.0, w3lib 1.19.0, Twisted 18.4.0, Python 3.6.4 | packaged by conda-forge | (default, Dec 23 2017, 16:31:06) - [GCC 4.8.2 20140120 (Red Hat 4.8.2-15)], pyOpenSSL 17.2.0 (OpenSSL 1.0.2n  7 Dec 2017), cryptography 2.0.3, Platform Linux-4.14.22+-x86_64-with-debian-stretch-sid
2018-07-02 04:00:47 [scrapy.crawler] INFO: Overridden settings: {}
2018-07-02 04:00:47 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2018-07-02 04:00:47 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloade